In [1]:
import requests
import zipfile
import shutil
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET 
from datetime import datetime 


In [2]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip'
response=requests.get(url)

In [3]:
if response.status_code == 200:
    # Save the content of the response to a file
    with open('source.zip', 'wb') as f:
        f.write(response.content)

    # Extract the contents of the zip file
    with zipfile.ZipFile('source.zip', 'r') as zip_ref:
        zip_ref.extractall('source')

    # Delete the zip file after extraction
    os.remove('source.zip')

    print("File downloaded and extracted successfully.")
else:
    print("Failed to download the file.")

File downloaded and extracted successfully.


In [4]:
directory = 'source'
os.listdir(directory)

['source1.csv',
 'source1.json',
 'source1.xml',
 'source2.csv',
 'source2.json',
 'source2.xml',
 'source3.csv',
 'source3.json',
 'source3.xml']

In [9]:
#These are transformed_data.csv, to store the final output data that you can load to a database, and log_file.txt, that stores all the logs.
#Introduce these paths in the code by adding the following statements:
log_file = "log_file.txt" 
target_file = "transformed_data.csv" 

In [7]:
#Function for extract the csv file
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [8]:
#Function for extract the json file
def extract_from_json(file_to_process): 
    dataframe = pd.read_json(file_to_process, lines=True) 
    return dataframe 

In [10]:
#Function for extract the xml file
def extract_from_xml(file_to_process): 
    dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        name = person.find("name").text
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True) 
    return dataframe

To call the relevant function, write a function extract, which uses the glob library to identify the filetype. This can be done as follows:

In [11]:
def extract(): 
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data 
     
    # process all csv files 
    for csvfile in glob.glob("*.csv"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True) 
         
    # process all json files 
    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True) 
     
    # process all xml files 
    for xmlfile in glob.glob("*.xml"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True) 
         
    return extracted_data 

In [13]:
#Transfered the extracted files
def transform(data): 
    '''Convert inches to meters and round off to two decimals 
    1 inch is 0.0254 meters '''
    data['height'] = round(data.height * 0.0254,2) 
 
    '''Convert pounds to kilograms and round off to two decimals 
    1 pound is 0.45359237 kilograms '''
    data['weight'] = round(data.weight * 0.45359237,2) 
    
    return data

In [14]:
def load_data(target_file, transformed_data): 
    transformed_data.to_csv(target_file) 

In [15]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

ETL operations and log progress

In [17]:
#ETL operations and log progress
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract()
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
 
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 
 
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
 
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
Empty DataFrame
Columns: [name, height, weight, Unnamed: 0]
Index: []
